In [2]:
import random
import pandas as pd
import numpy as np
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from transformers import BertModel

from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
model = BertModel.from_pretrained('kykim/bert-kor-base')

In [8]:
class CustomModel(nn.Module):
  def __init__(self, num_classes):
    super(CustomModel, self).__init__()
    
    # image
    self.image_model = models.efficientnet_b0(pretrained=True)
    for params in self.image_model.parameters():
      params.requires_grad = True
    self.image_model.classifier[1] = nn.Linear(in_features=1280, out_features=256)
    # self.linear = nn.Linear(1280, num_classes)
    # text
    self.text_model = BertModel.from_pretrained('kykim/bert-kor-base')
    self.dropout = nn.Dropout(0.3)

    # linear
    self.linear = nn.Linear(256+768, num_classes)
    # softmax function
    self.softmax = nn.Softmax()

  def forward(self, image, text, text_mask):
    # image result
    image_output = self.image_model(image)

    # text result
    _, text_output = self.text_model(text, attention_mask=text_mask, return_dict=False)
    text_output = self.dropout(text_output)

    # concat
    output = torch.cat([image_output, text_output], axis=1)
    output = self.linear(output)
    output = self.softmax(output)
    return output

In [9]:
model = CustomModel(120)

Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
